# Recommending songs based on Embeddings

In [31]:
import pandas as pd
from urllib import request
import numpy as np

In [21]:
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

In [22]:
lines = data.read().decode('utf-8').split('\n')[2:]

In [24]:
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 2]

In [27]:
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode('utf-8').split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(songs, columns=['song_id', 'song_name', 'artist'])
songs_df = songs_df.set_index('song_id')
songs_df.head()

,song_name,artist
song_id,,
0,Gucci Time (w\/ Swizz Beatz),Gucci Mane
1,Aston Martin Music (w\/ Drake & Chrisette Mich...,Rick Ross
2,Get Back Up (w\/ Chris Brown),T.I.
3,Hot Toddy (w\/ Jay-Z & Ester Dean),Usher
4,Whip My Hair,Willow


In [30]:
from gensim.models import Word2Vec

model = Word2Vec(
    playlists,
    vector_size=32,
    window=20,
    min_count=1,
    workers=4
)

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


In [37]:
model.wv.most_similar(positive=str(2172), topn=5)

[('6644', 0.9952674508094788),
 ('1999', 0.9943102598190308),
 ('1785', 0.9943013191223145),
 ('6658', 0.9937373995780945),
 ('1922', 0.9933514595031738)]

In [39]:
song_id = 2172

def get_song_embedding(song_id):
    similar_songs = np.array(
        model.wv.most_similar(
            positive=str(song_id),
            topn=5
        )
    )[:,0]
    return songs_df.iloc[similar_songs]


In [40]:
get_song_embedding(21)

,song_name,artist
song_id,,
27114,Blow The Whistle,Too Short
11653,"The Sweetest Girl (Dollar Bill) (w\/ Akon, Lil...",Wyclef Jean
8705,I Got It Made,Special Ed
5805,Power (w\/ Dwele),Kanye West
23416,Rebirth Of Slick (Cool Like Dat),Digable Planets


In [47]:
import gradio as gr

interface = gr.Interface(
    fn=get_song_embedding,
    inputs=gr.Number(label="Song ID"),
    outputs=gr.Dataframe(label="Similar Songs", scale=0),
    title="Song Recommendation System",
    description="Enter a song ID to get similar songs"
)

In [48]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/charmainemahachi/Hands-On-LLMs/venv/lib/python3.12/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/charmainemahachi/Hands-On-LLMs/venv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/charmainemahachi/Hands-On-LLMs/venv/lib/python3.12/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/charmainemahachi/Hands-On-LLMs/venv/lib/python3.12/site-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/charmainemahachi/Hands-On-L

Created dataset file at: .gradio/flagged/dataset1.csv
